In [129]:
# Data processing tool for Nova Home Support
# IPython Notebook for testing
#
# Import Python packages
# Note: We're only using packages that come with Anaconda standard distribution: https://www.anaconda.com/ 
import pandas as pd # For importing, manipulating, and exporting data
import re # Python regular expression support
import datetime # Python datetime conversion support
from concurrent.futures import process # Asynchronous callable execution
from dateutil.easter import *
import holidays
from tkinter import * # For user interface
from tkinter.ttk import * # UI widgets
from tkinter.filedialog import askopenfile, askopenfilename, askdirectory # UI-file system interaction
from copy import deepcopy
from collections import namedtuple

Range = namedtuple('Range', ['start', 'end'])

In [130]:
# Read in the dataset
df = pd.read_excel('shift.xlsx') 
# Keep useful columns
df = df[['Service 1 Description (Code)', 
         'Service Provider','Check-In Date',
         'Check-In Time',
         'Updated Check-In Date',
         'Updated Check-In Time',
         'Check-Out Date',
         'Check-Out Time',
         'Updated Check-Out Date',
         'Updated Check-Out Time',
         'Staff Worked Duration',
         'Staff Worked Duration (Minutes)']]

In [131]:
# Remove parentheses and everything within them
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].str.replace(r'\(.*\)', '')

# Remove prefix if it exists
prefix = 'RC-SDP-CLS-320 '
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].apply(lambda x: x[len(prefix):] if x.startswith(prefix) else x)
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].apply(lambda x: x.rstrip())

/var/folders/kl/0xlphzyd1450znbjp9zww9tw0000gn/T/ipykernel_43275/4018409904.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].str.replace(r'\(.*\)', '')


In [132]:
# Remove everything after " /"
df['Service Provider'] = df['Service Provider'].str.split(' /', n=1).str[0]

In [133]:
# Replace Date/Time with Updated Date/Time if the latter is not NaN
df['Check-In Date'] = df['Updated Check-In Date'].fillna(df['Check-In Date'])
df['Check-In Time'] = df['Updated Check-In Time'].fillna(df['Check-In Time'])
df['Check-Out Date'] = df['Updated Check-Out Date'].fillna(df['Check-Out Date'])
df['Check-Out Time'] = df['Updated Check-Out Time'].fillna(df['Check-Out Time'])
df.drop(['Updated Check-In Date', 'Updated Check-In Time','Updated Check-Out Date', 'Updated Check-Out Time'], axis=1, inplace=True)

In [134]:
# Create datetime series in Python format
CIDT = df['Check-In Date'].str.cat(df['Check-In Time'], sep=' ')
CODT = df['Check-Out Date'].str.cat(df['Check-Out Time'], sep=' ')
CIDT = CIDT.apply(lambda x: datetime.datetime.strptime(x, r'%m/%d/%Y %I:%M %p'))
CODT = CODT.apply(lambda x: datetime.datetime.strptime(x, r'%m/%d/%Y %I:%M %p'))
df['CIDT'] = CIDT
df['CODT'] = CODT

In [135]:
# Calculate time difference from check-in and check-out datetimes
CTD = (CODT - CIDT).dt.total_seconds() / 60
# Convert Staff Work Duration from Hour:Minutes to Minutes
SWD_min = df['Staff Worked Duration'].apply(lambda x: (int(x.split(':')[0]) * 60) + int(x.split(':')[1]))

In [136]:
# Sanity check:
# 1. Check if Staff Work Duration ==  Staff Work Duration (Minutes)
sanity1 = (SWD_min == df['Staff Worked Duration (Minutes)'])
# 2. Check if |Staff Work Duration (Minutes) - Calculated Time Difference| <= 1
sanity2 = ((df['Staff Worked Duration (Minutes)'] - CTD).abs() <= 1.1) # 1.1 to avoid float precision issues
df["Error Check"] = (sanity1 & sanity2) # The data is "sane" only when both checks are passed

In [137]:
#PAYROLL
df = df.rename(columns={'Service 1 Description (Code)': 'Shift Code'})
df_payroll = deepcopy(df.loc[~df['Shift Code'].str.contains('Adaptive Behavior Treatment')])
# split the 'Service Provider' column on comma separator and extract First name and Surname
df_payroll[['Surname', 'First Name']] = df_payroll['Service Provider'].str.split(', ', expand=True)
# concatenate First name and Surname columns in the desired order
df_payroll['Service Provider'] = df_payroll['First Name'] + ' ' + df_payroll['Surname']
# drop First name and Surname columns
df_payroll.drop(columns=['First Name', 'Surname'], inplace=True)
# Now process billing:
# Read "Manger Rates" sheet
manager_rates = pd.read_excel("billing.xlsx", sheet_name="Manager Rates")
# Read "Non-manager Rates" sheet
non_manager_rates = pd.read_excel("billing.xlsx", sheet_name="Non-manager Rates")
non_manager_rates = non_manager_rates[['Shift Code', 'Regular Hourly Wage', 'BOT Calculation', 'Hours Accrued']]
#merge:
df_payroll=pd.merge(df_payroll, non_manager_rates, how='left', on='Shift Code')

In [144]:
def approved_holiday(YEARS):
    approved_holiday = set()
    holiday_set = set(["Thanksgiving", "Christmas Day"])
    for year in YEARS:
        ah = {k for k, v in holidays.US(years=year).items() if v in holiday_set}
        approved_holiday = approved_holiday.union(ah)
        approved_holiday.add(easter(year))
        approved_holiday.add(datetime.date(year, 12, 24))
        approved_holiday.add(datetime.date(year, 12, 31))
        approved_holiday.add(datetime.date(year-1, 12, 31))
    return approved_holiday

def approved_holiday_hours(YEARS):
    approved_holiday_dt = []
    for x in approved_holiday(YEARS):
        if (x.month == 12 and x.day == 31):
            approved_holiday_dt.append(Range(start = datetime.datetime.combine(x, datetime.time(hour=23)),end = datetime.datetime.combine(x+datetime.timedelta(days=1), datetime.time(hour=23))))
        else:
            approved_holiday_dt.append(Range(start = datetime.datetime.combine(x, datetime.time(hour=7)), end = datetime.datetime.combine(x, datetime.time(hour=23))))
    return approved_holiday_dt

def work_holiday_overlap(work_range, ahh_list):
    total_overlap = 0.0
    for ahh in ahh_list:
        latest_start = max(work_range.start, ahh.start)
        earliest_end = min(work_range.end, ahh.end)
        delta = (earliest_end - latest_start)
        overlap = max(0, delta.total_seconds())
        total_overlap += overlap
    return total_overlap/60 #in minutes

def calc_holiday_hours(df_payroll):
        years = set([min([df_payroll['CIDT'][i].year for i in range(len(df_payroll))]), max([df_payroll['CODT'][i].year for i in range(len(df_payroll))])])
        duration_range = [Range(start = df_payroll['CIDT'][i].to_pydatetime(), end = df_payroll['CODT'][i].to_pydatetime()) for i in range(len(df_payroll))]
        ahh_list = approved_holiday_hours(years)
        holiday_hours = [work_holiday_overlap(duration_range[i], ahh_list) for i in range(len(df_payroll))]
        df_payroll['Holiday Hours'] = holiday_hours
        
def is_manager(name, manager_rates):
    return name in set(manager_rates['Name'])

def manager_is_exempt(name, df_payroll):
    exempt_mins = df_payroll.loc[(df_payroll['Service Provider'] == name) & (df_payroll['Shift Code'] == 'MGR Direct Care')]['Staff Worked Duration (Minutes)'].sum()
    non_exempt_mins = df_payroll.loc[(df_payroll['Service Provider'] == name) & (df_payroll['Shift Code'] != 'MGR Direct Care')]['Staff Worked Duration (Minutes)'].sum()
    return (exempt_mins >= non_exempt_mins)

def worked_overtime(name, df_payroll):
    return df_payroll.loc[df_payroll['Service Provider'] == name]['Staff Worked Duration (Minutes)'].sum() > (40*60)
